<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2017/Chapter_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
type(5)

<class 'int'>


In [ ]:
type(['hello', 'goodbye'])

<class 'list'>


In [ ]:
type_result = type(5)
type(type_result)

<class 'type'>


In [ ]:
type("Hello") == type("Goodbye")

True


In [ ]:
type("Hello") == type(5)

False


In [ ]:
class A:
    pass

class B(A):
    pass


In [ ]:
b = B()
type(b)

<class '__main__.B'>


In [ ]:
b.__class__

<class '__main__.B'>


In [ ]:
b_class = b.__class__
b_class == B

True


In [ ]:
b_class.__name__

'B'


In [ ]:
b_class.__bases__

(<class '__main__.A'>,)


In [ ]:
class C:
    pass

class D:
    pass

class E(D):
    pass

x = 12
c = C()
d = D()
e = E()
isinstance(x, E)

False


In [ ]:
isinstance(c, E)             #A

False


In [ ]:
isinstance(e, E)

True


In [ ]:
isinstance(e, D)             #B

True


In [ ]:
isinstance(d, E)                #C

False


In [ ]:
y = 12
isinstance(y, type(5))             #D

True


In [ ]:
issubclass(C, D)

False


In [ ]:
issubclass(E, D)

True


In [ ]:
issubclass(D, D)           #E

True


In [ ]:
issubclass(e.__class__, D)

True


In [ ]:
class Color:
    def __init__(self, red, green, blue):
        self._red = red
        self._green = green
        self._blue = blue
    def __str__(self):
        return "Color: R={0:d}, G={1:d}, B={2:d}".format (self._red,
                                             self._green, self._blue)

In [ ]:
from color_module import Color
c = Color(15, 35, 3)

In [ ]:
print(c)

Color: R=15, G=35, B=3


In [ ]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)                     #A
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                          "be a list.")
        for element in initial_list:
                if not isinstance(element, self.type):
                    raise TypeError("Attempted to add an element of "
                                  "incorrect type to a typed list.")
        self.elements = initial_list[:]

In [ ]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        self.elements = initial_list[:]
    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.elements[i] = element
    def __getitem__(self, i):
        return self.elements[i]

In [ ]:
x = TypedList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')


In [ ]:
class TypedListList(list):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")

    def __setitem__(self, i, element):
        self.__check(element)
        super().__setitem__(i, element)

In [ ]:
x = TypedListList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')


In [ ]:
x[:]

['', '', 'Hello', 'There', '']


In [ ]:
del x[2]
x[:]

['', '', 'There', '']


In [ ]:
x.sort()
x[:]

['', '', '', 'There']


In [ ]:
from collections import UserList
class TypedUserList(UserList):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.data[i] = element
    def __getitem__(self, i):
        return self.data[i]

In [ ]:
x = TypedUserList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')


In [ ]:
x[:]

['', '', 'Hello', 'There', '']


In [ ]:
del x[2]
x[:]

['', '', 'There', '']


In [ ]:
x.sort()
x[:]

['', '', '', 'There']
